# EDA Notebook

In [1]:
# imports
import wandb
import pandas as pd

In [2]:
# fetch the sample.csv file from W&B and create a dataframe
run = wandb.init(project="nyc_airbnb", group="eda", save_code=True)
local_path = wandb.use_artifact("sample.csv:latest").file()
df = pd.read_csv(local_path)

wandb: Currently logged in as: frizzles7 (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.1 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [3]:
# create a profile
import pandas_profiling

profile = pandas_profiling.ProfileReport(df)
profile.to_widgets()

Summarize dataset:   0%|          | 0/29 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

## Review of profile

In the profile, I notice the following things:
- there are no duplicate rows in the data
- there are some missing values in last_review and reviews_per_month
- there are a lot of zeros in number_of_reviews and availability_365

Reviewing the individual columns, I note the following:
- in neighbourhood_group, most observations are in Manhattan and Brooklyn
- in prices, there a some zeros (free rooms?) and prices as high as 10,000 (that seems high, even for NYC)
- in minimum_nights, the max is 1250, which is almost 3.5 years (also seems unreasonable)
- in last_review, the data stored is a date, but this is currently stored as a string

In [4]:
# drop outliers on price
# min and max prices provided by stakeholders of $10 and $350
min_price = 10
max_price = 350
idx = df['price'].between(min_price, max_price)
df = df[idx].copy()

In [5]:
# convert last_review column from string to datetime
df['last_review'] = pd.to_datetime(df['last_review'])

In [6]:
# drop outliers on minimum_nights
min_nights = 1
max_nights = 100
idx = df['minimum_nights'].between(min_nights, max_nights)
df = df[idx].copy()

In [7]:
# confirm issues have been resolved
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18935 entries, 0 to 19999
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   id                              18935 non-null  int64         
 1   name                            18928 non-null  object        
 2   host_id                         18935 non-null  int64         
 3   host_name                       18927 non-null  object        
 4   neighbourhood_group             18935 non-null  object        
 5   neighbourhood                   18935 non-null  object        
 6   latitude                        18935 non-null  float64       
 7   longitude                       18935 non-null  float64       
 8   room_type                       18935 non-null  object        
 9   price                           18935 non-null  int64         
 10  minimum_nights                  18935 non-null  int64         
 11  nu

In [8]:
df.describe()

,id,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
count,1.893500e+04,1.893500e+04,18935.000000,18935.000000,18935.000000,18935.000000,18935.000000,15207.000000,18935.000000,18935.000000
mean,1.884672e+07,6.646012e+07,40.728054,-73.950770,122.298020,6.034117,23.840665,1.383227,6.570372,109.370848
std,1.096344e+07,7.783999e+07,0.055407,0.046863,71.509451,9.805422,45.508643,1.690770,31.120172,130.410700
min,2.539000e+03,2.571000e+03,40.508730,-74.239140,10.000000,1.000000,0.000000,0.010000,1.000000,0.000000
25%,9.371109e+06,7.763216e+06,40.688820,-73.981990,66.000000,1.000000,1.000000,0.190000,1.000000,0.000000
50%,1.941018e+07,3.052348e+07,40.721710,-73.954570,100.000000,2.000000,6.000000,0.730000,1.000000,39.000000
75%,2.891999e+07,1.048749e+08,40.763220,-73.934430,160.000000,5.000000,24.000000,2.010000,2.000000,218.000000
max,3.648561e+07,2.742733e+08,40.913060,-73.717950,350.000000,100.000000,607.000000,27.950000,327.000000,365.000000


In [9]:
# finish the run
run.finish()